In [13]:
import csv
import itertools
import re
import pandas as pd


#* ENVS
wspr_csv = "0b19055bea652ab7d3f20f40b15eb973.csv"
time_slot = 4
re_time_slot = f'^..:.{time_slot}'
parsed_csv = f'parsed_{wspr_csv}'

header = []
parsed_dt = []
alt = []


def alt_calc(power,split_ts):
    if split_ts == None:
        alt_m = int(power)*20 #* Type 3 WSPR message decode x 20 (20m per dBm)
    else:
        alt_m = int(power)*300 #* Type 1 or 2 WSPR message decode x 300 (300m per dBm)
    alt_ft = alt_m * 3.28 #* Convert meters to feet
    return (alt_m, int(alt_ft))


with open(wspr_csv, 'r') as file:
    data = csv.reader(file)
    header = next(data)
    for row in data:
        alt = []
        split_dt = row[1].split(' ')
        split_ts = re.search(re_time_slot, split_dt[1])
        alt_m = alt_calc(row[15], split_ts)
        alt.extend(alt_m)
        parsed_dt.append(list(itertools.chain(row, split_dt, alt)))

add_headers = ['date', 'time', 'altitude_m', 'altitude_ft']
header.extend(add_headers)

with open(parsed_csv, "w",newline="") as f:
    writer = csv.writer(f)
    header[1] = 'datetime'
    # print(header)
    writer.writerow(header)
    for row in parsed_dt:
        writer.writerow(row)

In [14]:
#* Adding in a dataframe to work to add the WSPR 3 packet to the Type 1 and 2

parsed_df = pd.read_csv(parsed_csv, parse_dates=["datetime"], usecols=['datetime','date','time', 'tx_lat', 'tx_lon', 'tx_loc', 'altitude_m', 'altitude_ft'])
parsed_df.tail()

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft
2005,2022-01-29 13:54:00,59.479,-7.042,IO69,2022-01-29,13:54:00,8100,26568
2006,2022-01-29 13:54:00,59.479,-7.042,IO69,2022-01-29,13:54:00,8100,26568
2007,2022-01-29 13:54:00,59.479,-7.042,IO69,2022-01-29,13:54:00,8100,26568
2008,2022-01-29 13:54:00,59.479,-7.042,IO69,2022-01-29,13:54:00,8100,26568
2009,2022-01-29 13:54:00,59.479,-7.042,IO69,2022-01-29,13:54:00,8100,26568


In [15]:
parsed_df.dtypes

datetime       datetime64[ns]
tx_lat                float64
tx_lon                float64
tx_loc                 object
date                   object
time                   object
altitude_m              int64
altitude_ft             int64
dtype: object

In [16]:
parsed_df['type'] = parsed_df['time'].str[4]

In [17]:
parsed_df.drop_duplicates(subset ="datetime", keep = False, inplace = True)
parsed_df.shape

(19, 9)

In [18]:
parsed_df

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft,type
499,2022-01-27 13:34:00,39.479,-73.042,FM39,2022-01-27,13:34:00,8100,26568,4
500,2022-01-27 13:36:00,39.271,-73.958,FM39ag,2022-01-27,13:36:00,540,1771,6
685,2022-01-27 14:54:00,39.271,-73.208,FM39jg,2022-01-27,14:54:00,8100,26568,4
776,2022-01-27 15:34:00,39.271,-72.792,FM39og,2022-01-27,15:34:00,8100,26568,4
777,2022-01-27 15:36:00,39.313,-72.625,FM39qh,2022-01-27,15:36:00,600,1967,6
827,2022-01-27 15:54:00,39.313,-72.542,FM39rh,2022-01-27,15:54:00,8100,26568,4
828,2022-01-27 15:56:00,39.313,-72.458,FM39sh,2022-01-27,15:56:00,600,1967,6
865,2022-01-27 16:14:00,39.313,-72.375,FM39th,2022-01-27,16:14:00,8100,26568,4
866,2022-01-27 16:16:00,39.354,-72.292,FM39ui,2022-01-27,16:16:00,540,1771,6
1623,2022-01-27 20:54:00,39.938,-69.958,FM59aw,2022-01-27,20:54:00,8100,26568,4


In [19]:
parsed_df.to_csv('new_wspr.csv', index = False)

In [20]:
# I only print for type 0 packets and add the 0 detail to type 8 base.
with open("new_wspr.csv", "r") as a_file:
    for line in a_file:
        spline = line.split(',')
        # print(spline[8].strip('\n'))
        if spline[8].strip('\n') == '4':
            # print("it's an 8")
            base_altitude = int(spline[7])
        elif spline[8].strip('\n') == '6':
            altitude = base_altitude + int(spline[7])
            print(f"{spline[4]} {spline[5]} {altitude} ft")

2022-01-27 13:36:00 28339 ft
2022-01-27 15:36:00 28535 ft
2022-01-27 15:56:00 28535 ft
2022-01-27 16:16:00 28339 ft
2022-01-28 14:16:00 19352 ft
2022-01-28 14:36:00 17843 ft
2022-01-28 14:56:00 18040 ft
2022-01-29 11:56:00 27683 ft
2022-01-29 12:16:00 27683 ft
2022-01-29 13:06:00 27683 ft
2022-01-29 13:26:00 27683 ft


Now want to take the altitude in the :08 mins and add to the following :00 mins. to get teh precise altitude.

```
date,time,altitude_m,altitude_ft
2022-01-07,23:38:00,6900,22632
2022-01-07,23:40:00,1060,3476
```

For example the precise altitude is `7960 m`